In [1]:
import plotly.express as px
import pandas as pd
import numpy as np

from ostk.core.filesystem import Directory

from ostk.mathematics.geometry.d3.objects import Cuboid
from ostk.mathematics.geometry.d3.objects import Composite
from ostk.mathematics.geometry.d3.objects import Point

from ostk.physics import Environment
from ostk.physics.coordinate import Frame
from ostk.physics.environment.atmospheric import Earth as EarthAtmosphericModel
from ostk.physics.environment.gravitational import Earth as EarthGravitationalModel
from ostk.physics.environment.magnetic import Earth as EarthMagneticModel
from ostk.physics.environment.objects.celestial_bodies import Earth, Moon, Sun
from ostk.physics.time import DateTime, Duration, Instant, Scale, Time, Interval
from ostk.physics.units import Mass, Length, Angle

from ostk.astrodynamics.trajectory.state import NumericalSolver
from ostk.astrodynamics.trajectory.orbit.models.brouwerLyddaneMean import (
    BrouwerLyddaneMeanShort
)
from ostk.astrodynamics.trajectory import StateBuilder
from ostk.astrodynamics.trajectory.state import CoordinatesBroker, CoordinatesSubset
from ostk.astrodynamics.trajectory.state.coordinates_subset import CartesianPosition, CartesianVelocity
from ostk.astrodynamics.trajectory import Orbit, Sequence
from ostk.astrodynamics import Dynamics
from ostk.astrodynamics.event_condition import LogicalCondition, RealCondition
from ostk.astrodynamics.dynamics import Thruster
from ostk.astrodynamics.flight.system import SatelliteSystem, PropulsionSystem
from ostk.astrodynamics.guidance_law import QLaw, QLawParameters
from ostk.astrodynamics.solvers import FiniteDifferenceSolver
from ostk.astrodynamics.trajectory.orbit.models import Tabulated
from ostk.astrodynamics.trajectory.orbit.models.kepler import COE

## User inputs

In [2]:
earth = Earth.from_models(
    EarthGravitationalModel(EarthGravitationalModel.Type.EGM96, Directory.undefined(), 20, 20),
    EarthMagneticModel(EarthMagneticModel.Type.Undefined),
    EarthAtmosphericModel(EarthAtmosphericModel.Type.NRLMSISE00),
)

environment = Environment(Instant.J2000(), [earth, Sun.default(), Moon.default()])

In [3]:
instant = Instant.date_time(DateTime(2023,1,1), Scale.UTC)
initial_state = Orbit.sun_synchronous(instant, Length.kilometers(505.0), Time.midnight(), Earth.default()).get_state_at(instant)

In [4]:
mass = Mass.kilograms(200.0)
wet_mass = Mass.kilograms(14.0)
propulsion_system = PropulsionSystem(thrust_si_unit=14.3e-3, specific_impulse_si_unit=1016.0)

## Setup environment, initial state and Satellite System

In [5]:
satellite_geometry = Composite(Cuboid(
    Point(0.0, 0.0, 0.0), np.eye(3).tolist(), [1.0, 0.0, 0.0]
))
satellite_system = SatelliteSystem(mass, satellite_geometry, np.eye(3), 2.0, 2.2, propulsion_system)

In [6]:
coordinates_broker = CoordinatesBroker(
    [
        CartesianPosition.default(),
        CartesianVelocity.default(),
        CoordinatesSubset.mass(),
        CoordinatesSubset.surface_area(),
        CoordinatesSubset.drag_coefficient(),
    ]
)

state_builder = StateBuilder(
    frame=Frame.GCRF(),
    coordinates_subsets=[
        CartesianPosition.default(),
        CartesianVelocity.default(),
        CoordinatesSubset.mass(),
        CoordinatesSubset.surface_area(),
        CoordinatesSubset.drag_coefficient(),
    ],
)

coordinates = [
    *initial_state.get_coordinates().tolist(),
    mass.in_kilograms() + wet_mass.in_kilograms(),
    satellite_system.get_cross_sectional_surface_area(),
    satellite_system.get_drag_coefficient()
]

state = state_builder.build(initial_state.get_instant(), coordinates)

In [7]:
dynamics = Dynamics.from_environment(environment)
# state_logger = lambda state: print(BrouwerLyddaneMeanShort.cartesian((state.get_position(), state.get_velocity()), EarthGravitationalModel.EGM2008.gravitational_parameter).get_semi_major_axis() - EarthGravitationalModel.EGM2008.equatorial_radius)
state_logger = None
numerical_solver = NumericalSolver.conditional(5.0, 1e-12, 1e-12, state_logger)

### Sequence

In [8]:
sequence = Sequence(
    repetition_count=20,
    numerical_solver=numerical_solver,
    dynamics=dynamics,
    maximum_propagation_duration=Duration.days(30.0),
    verbosity=5
)

sma_evaluator = lambda state: BrouwerLyddaneMeanShort.cartesian((state.get_position(), state.get_velocity()), EarthGravitationalModel.EGM2008.gravitational_parameter).get_semi_major_axis().in_meters()

# burn till at 500.0
sma_thrust_condition = RealCondition(
    name="Mean SMA crossing - thrust",
    criterion=RealCondition.Criterion.StrictlyPositive,
    evaluator=sma_evaluator,
    target_value=EarthGravitationalModel.EGM2008.equatorial_radius.in_meters() + 500.0e3,
)

# duration condition
duration_condition = RealCondition.duration_condition(
    criterion=RealCondition.Criterion.StrictlyPositive,
    duration=Duration.minutes(15.0),
)

# Increasing the COE target SMA as it's osculating
target_coe = COE(
    EarthGravitationalModel.EGM2008.equatorial_radius + Length.meters(530.0e3),
    0.0,
    Angle.degrees(0.0),
    Angle.degrees(0.0),
    Angle.degrees(0.0),
    Angle.degrees(0.0)
)

guidance_law = QLaw(
    target_coe,
    EarthGravitationalModel.EGM2008.gravitational_parameter,
    QLawParameters(
        element_weights={COE.Element.SemiMajorAxis: 1.0, COE.Element.Eccentricity: 1.0},
        m=3,
        n=4,
        r=2,
        b=0.01,
    ),
    FiniteDifferenceSolver.default(),
)

sequence.add_maneuver_segment(
    LogicalCondition("test", LogicalCondition.Type.Or, [duration_condition, sma_thrust_condition]),
    Thruster(
        satellite_system=satellite_system,
        guidance_law=guidance_law,
    ),
)

sequence.add_coast_segment(
    RealCondition.duration_condition(criterion=RealCondition.Criterion.StrictlyPositive, duration=Duration.hours(2.0))
)
    

In [ ]:
sol = sequence.solve(state, event_condition=sma_thrust_condition)
states = sol.get_states()

In [10]:
model = Tabulated(states, 0, Tabulated.InterpolationType.BarycentricRational)

In [11]:
orbit = Orbit(model, environment.access_celestial_object_with_name("Earth"))

In [12]:
interval = Interval.closed(states[0].get_instant(), states[-1].get_instant())
interpolated_states = orbit.get_states_at(interval.generate_grid(Duration.seconds(60.0)))

In [13]:
data = []
for state in interpolated_states:
    blmshort = BrouwerLyddaneMeanShort.cartesian((state.get_position(), state.get_velocity()), earth.get_gravitational_parameter())
    data.append(
        {
            "altitude": float(blmshort.get_semi_major_axis().in_kilometers() - earth.get_equatorial_radius().in_kilometers()),
            "eccentricity": float(blmshort.get_eccentricity()),
            "mass": float(state.get_coordinates()[6]),
            "time": state.get_instant().get_date_time(Scale.UTC)
        }
    )
    

In [14]:
df = pd.DataFrame(data)

In [ ]:
fig = px.scatter(df, x="time", y="altitude", color="eccentricity")
for segment_solution in sol.segment_solutions:
    time = segment_solution.states[-1].get_instant().get_date_time(Scale.UTC)
    fig.add_vline(
        x=time.isoformat()
    )
fig.show()

In [16]:
fig = px.scatter(df, x="time", y="eccentricity")
for segment_solution in sol.segment_solutions:
    time = segment_solution.states[-1].get_instant().get_date_time(Scale.UTC)
    fig.add_vline(
        x=time.isoformat()
    )
fig.show()

In [17]:
fig2 = px.line(df, x="time", y="mass")
for segment_solution in sol.segment_solutions:
    time = segment_solution.states[-1].get_instant().get_date_time(Scale.UTC)
    fig2.add_vline(
        x=time.isoformat()
    )
fig2.show()